<a href="https://colab.research.google.com/github/JohnNun/pneumonia_identification_project/blob/main/pneumonia_identification_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pneumonia Identification Project

# Business and Data Understanding


# Data preparation

In [ ]:
# For Google Colab only
!pip install keras
!pip install keras_preprocessing
!pip install tensorflow
!pip install scikeras[tensorflow]

In [2]:
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
import os

In [3]:
# Following code taken from stackoverflow user fdsig
import json
import os
from pathlib import Path

# your api key
api_key = {
'username':"" ,
'key':""}

# uses pathlib Path
kaggle_path = Path('/root/.kaggle')
os.makedirs(kaggle_path, exist_ok=True)

# opens file and dumps python dict to json object
with open (kaggle_path/'kaggle.json', 'w') as handl:
    json.dump(api_key,handl)

os.chmod(kaggle_path/'kaggle.json', 600)

In [ ]:
# Downloading and Importing images
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia

In [5]:
# Linking directory path
train_data_dir = 'chest_xray/train'
val_data_dir = 'chest_xray/val'

In [6]:
# Getting train and validation data from directory
test_generator = ImageDataGenerator().flow_from_directory(val_data_dir, target_size=(150,150), batch_size=16)
train_generator = ImageDataGenerator().flow_from_directory(train_data_dir, target_size=(150,150), batch_size=652)

Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [7]:
# Creating datasets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

In [34]:
# Reshaping data
train_img_unrow = train_images.reshape(652, -1)
test_img_unrow = test_images.reshape(16, -1)

In [35]:
# Normalizing data
train_img_final = train_img_unrow/255
test_img_final = test_img_unrow/255

In [42]:
train_img_final[1]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

# Modeling

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

In [37]:
model = Sequential()

In [49]:
model.add(Dense(50, activation='relu', input_dim=67500))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [50]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
model.fit(train_img_final, train_labels, epochs=100, validation_data=(test_img_final, test_labels))

Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.7069 - loss: 0.6466 - val_accuracy: 0.5000 - val_loss: 0.7439
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.7185 - loss: 0.6122 - val_accuracy: 0.5000 - val_loss: 0.7726
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.7255 - loss: 0.5948 - val_accuracy: 0.5000 - val_loss: 0.7894
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.7315 - loss: 0.5834 - val_accuracy: 0.5000 - val_loss: 0.8086
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.7382 - loss: 0.5758 - val_accuracy: 0.5000 - val_loss: 0.7980
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.7146 - loss: 0.5985 - val_accuracy: 0.5000 - val_loss: 0.8056
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.7132 - loss: 0.5997 - val_accuracy: 0.5000 - val_loss: 0.7986
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - accuracy: 0.6861 - loss: 0.6249 - val_accuracy: 0.

# Evaluation